In [ ]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.externals.joblib import dump, load

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
## read the data
df_data = pd.read_csv('/content/drive/MyDrive/cicids_static_data.csv')

In [ ]:
## check the 1st 5 rows
df_data.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,53,159,2,2,68,266,34,34,34.0,0.0,133,133,133.0,0.0,2.100629e+06,25157.232700,53.00000,90.066642,157,1,1,1.0,0.0,1,1,1,1.0,0.0,1,1,0,0,0,0,40,40,12578.616350,12578.616350,34,133,73.6,54.224533,2940.3,0,0,0,0,0,0,0,0,1,92.00,34.0,133.0,40,0,0,0,0,0,0,2,68,2,266,-1,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,53,51673,2,2,90,166,45,45,45.0,0.0,83,83,83.0,0.0,4.954231e+03,77.409866,17224.33333,29827.358290,51666,3,4,4.0,0.0,4,4,3,3.0,0.0,3,3,0,0,0,0,64,64,38.704933,38.704933,45,83,60.2,20.813457,433.2,0,0,0,0,0,0,0,0,1,75.25,45.0,83.0,64,0,0,0,0,0,0,2,90,2,166,-1,-1,1,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,7496,43,1,1,0,6,0,0,0.0,0.0,6,6,6.0,0.0,1.395349e+05,46511.627910,43.00000,0.000000,43,43,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,40,20,23255.813950,23255.813950,0,6,2.0,3.464102,12.0,0,0,0,1,0,0,0,0,1,3.00,0.0,6.0,40,0,0,0,0,0,0,1,0,1,6,29200,0,0,40,0.0,0.0,0,0,0.0,0.0,0,0,ATTACK
3,46564,61,1,1,6,6,6,6,6.0,0.0,6,6,6.0,0.0,1.967213e+05,32786.885250,61.00000,0.000000,61,61,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,16393.442620,16393.442620,6,6,6.0,0.000000,0.0,0,0,0,0,1,1,0,0,1,9.00,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,440,307,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,80,69,2,0,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.000000e+00,28985.507250,69.00000,0.000000,69,69,69,69.0,0.0,69,69,0,0.0,0.0,0,0,0,0,0,0,40,0,28985.507250,0.000000,0,0,0.0,0.000000,0.0,0,0,0,0,1,0,0,0,0,0.00,0.0,0.0,40,0,0,0,0,0,0,2,0,0,0,251,-1,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [ ]:
len(df_data)

25191

In [ ]:
## some stats about the data
df_data.describe()
## From this we can drop the columns with only zeros: FWD

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,25191.000000,2.519100e+04,25191.000000,25191.00000,2.519100e+04,2.519100e+04,25191.000000,25191.000000,25191.000000,25191.000000,25191.00000,25191.000000,25191.000000,25191.000000,2.518500e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,25191.000000,25191.0,25191.000000,25191.0,2.519100e+04,2.519100e+04,2.519100e+04,25191.000000,25191.000000,25191.000000,25191.000000,25191.000000,2.519100e+04,25191.000000,25191.000000,25191.000000,25191.000000,25191.000000,25191.000000,25191.000000,25191.000000,25191.000000,25191.000000,25191.000000,25191.000000,2.519100e+04,25191.0,25191.0,25191.0,25191.0,25191.0,25191.0,25191.000000,2.519100e+04,25191.00000,2.519100e+04,25191.000000,25191.000000,25191.000000,25191.000000,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04,2.519100e+04
mean,8765.751379,1.302190e+07,14.372752,16.80628,5.837956e+02,2.992545e+04,219.933746,20.452860,62.091345,72.396546,624.78143,45.731333,231.531587,227.162493,inf,inf,1.139005e+06,2.295824e+06,6.924351e+06,1.629789e+05,1.269634e+07,2.327944e+06,2.246253e+06,6.762395e+06,1.163676e+06,1.002364e+07,1.843998e+06,1.244500e+06,4.274531e+06,1.092621e+06,0.048787,0.0,0.000079,0.0,3.314755e+02,3.722468e+02,5.995505e+04,6893.287051,18.223850,713.829423,139.739549,220.086914,2.957633e+05,0.026597,0.048787,0.000159,0.280378,0.302013,0.105831,0.000079,0.000159,0.687785,157.253539,62.091345,231.531587,3.314755e+02,0.0,0.0,0.0,0.0,0.0,0.0,14.372752,5.837956e+02,16.80628,2.992641e+04,7068.153348,2338.379024,10.750308,25.890358,8.175780e+04,4.553670e+04,1.631345e+05,5.662674e+04,6.095297e+06,3.628463e+05,6.366900e+06,5.795319e+06
std,19077.629319,3.191602e+07,1220.476270,1620.27646,7.757115e+03,3.632427e+06,739.251288,60.512982,191.647090,293.360718,1481.65387,70.516212,472.544033,624.163182,NaN,NaN,4.418710e+06,7.231620e+06,2.035185e+07,2.809065e+06,3.181629e+07,9.748498e+06,7.573190e+06,2.040783e+07,9.181693e+06,2.889146e+07,9.324723e+06,5.304017e+06,1.565741e+07,8.915008e+06,0.215427,0.0,0.008910,0.0,2.493288e+04,3.240846e+04,2.337848e+05,39174.193402,26.166304,1606.101337,252.870523,497.329648,1.240815e+06,0.160905,0.215427,0.012600,0.449193,0.459140,0.307628,0.008910,0.012600,0.666127,273.365361,191.647090,472.544033,2.493288e+04,0.0,0.0,0.0,0.0,0.0,0.0,1220.476270,7.757115e+03,1620.27646,3.632433e+06,14824.236344,9227.293545,1179.682409,6.436242,8.378623e+05,4.001228e+05,1.208254e+06,7.917691e+05,1.949681e+07,3.767660e+06,2.013473e+07,1.923531e+07
min,0.000000,-1.000000e+00,1.000000,0.00000,0.000000e+

In [ ]:
## get names of columns of data
df_data.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

In [ ]:
## get the columns with null values greater than zero
df_data.columns[df_data.isna().sum() >0]

Index(['Flow Bytes/s'], dtype='object')

In [ ]:
## check if this column has really missing values
df_data['Flow Bytes/s'].isna().sum()

6

In [ ]:
"""
Here there was a problem in dropping the infinity values; even though I'd drop them they persist and keep being there: some reasons:
1.  There might be no nulls but 'np.inf' values
2.   Check from the describe if there are any inf values for different columns (as these won't appear in conditions of nulls)
3.   Drop by condition better than the index as this might be tricky; condition gets it at once with inplace = True
4.   I'm not sure if this affected but make a copy for the dataframe to drop on
"""

In [ ]:
## Drop nulls ==> this drops the nulls in flow/bytes
## Since the dataframe is of a quite large number we can easily drop the values ((we don't need to know the column or anything; the above trials was for us))
## But we need to remove it from the whole dataframe
df_data_na = df_data_copy.dropna()

In [ ]:
## length should be less by 6
len(df_data_na)

25185

In [ ]:
# Here we drop the infinity values in Flow Bytes
df_data_na.drop(df_data_na[df_data_na['Flow Bytes/s'] == np.inf].index, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
## Recheck
idx = df_data_na.index[df_data_na['Flow Bytes/s'] == np.inf]
idx = list(idx)
print(idx, df_data_na['Flow Bytes/s'].isna().sum())

[] 0


In [ ]:
## we can see the range of values (-1.2x10^-7 to 1.86*10^5) if we want to replace the NaN by mean value: the value was fixed
df_data_na['Flow Bytes/s'].describe()

count    2.517900e+04
mean     1.661845e+06
std      2.785234e+07
min     -1.200000e+07
25%      1.194084e+02
50%      4.713444e+03
75%      1.860465e+05
max      2.071000e+09
Name: Flow Bytes/s, dtype: float64

In [ ]:
### Now we need to remove the infinity values in Flow Packets ==> we can see there are no inf values
idx2 = df_data_na.index[df_data_na[' Flow Packets/s'] == np.inf]
idx2

Int64Index([], dtype='int64')

In [ ]:
df_data_na[' Flow Packets/s'].describe() ## And its describe is cleaned now

count    2.517900e+04
mean     6.680076e+04
std      2.420045e+05
min     -2.000000e+06
25%      6.514248e+00
50%      1.070754e+02
75%      2.209945e+04
max      3.000000e+06
Name:  Flow Packets/s, dtype: float64

In [ ]:
# ## get the indices of infinity in flow packets
# idx = df_data_na.index[df_data_na[' Flow Packets/s'] == np.inf]
# ## here we can see there're inf values
# df_data_na[' Flow Packets/s'].value_counts().sort_index()


In [ ]:
## check the number of rows in dataframe
len(df_data_na)

25179

In [ ]:
## get the data
df_x = df_data_na.iloc[:, :-1]

In [ ]:
## get the label of the data
df_y = df_data_na.iloc[:, -1]

In [ ]:
# scaler = StandardScaler()
# df_x = scaler.fit_transform(df_x)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df_x, df_y, test_size=0.3, random_state=42)

In [ ]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0).fit(X_train, y_train)

In [ ]:
score = clf.score(X_test, y_test)

In [ ]:
print(score)

0.9899391051098756


In [ ]:
#### For more finetuning we may use stacking for Random Forest, xgboost and svm

In [ ]:
### Save the standard scalar; can be done via pickle
## https://stackoverflow.com/questions/53152627/saving-standardscaler-model-for-use-on-new-datasets
# dump(scaler, 'std_scaler.bin', compress=True)

['std_scaler.bin']

In [ ]:
dump(clf, 'model.bin', compress=True)

['model.bin']

In [ ]:
# sc=load('std_scaler.bin')
model = load('model.bin')